# Optimization

1. Don't do it.
2. Only do it if you can measure the quality you want to optimize
3. Don't do it yet.
4. Only do it if you record your original performance.
5. Make sure you're optimizing what you want optimized
6. Record original performance
7. Investigate performance
8. Change system
9. Measure new performance
10. Compare
11. If not enough goto 6.

Here's our program. 

* It generates a mapping of symbols to notes
* Converts a document of symbols into notes. 
* Writes out the decrypted document to disk

![Dorabella Cipher](Dorabella-cipher-image.png)

Our idea is the direction and bumps are meaningful so we make a symbolic encoding where direction and bumps are encoded.

⇐2 ⇒3 ⇖2 ⇐3 ⇐1 ⇑2 ⇓1 ⇐3 ⇗1 ⇙2 ⇖3 ⇘2 ⇘1 ⇖1 ⇘2 ⇑3 ⇘2 ⇘2 ⇑2 ⇒3 ⇒3 ⇘2 ⇖1 ⇙1 ⇙2 ⇙1 ⇑1 ⇖3 ⇘3 

⇓1 ⇘2 ⇓1 ⇑2 ⇙1 ⇐3 ⇗1 ⇗2 ⇐3 ⇖2 ⇘2 ⇘2 ⇖2 ⇑2 ⇖1 ⇘1 ⇓1 ⇘2 ⇖3 ⇘2 ⇑2 ⇓2 ⇘3 ⇘1 ⇖1 ⇙1 ⇗1 ⇗1 ⇙1 ⇖3 ⇘3 

⇖2 ⇘3 ⇑2 ⇓2 ⇘3 ⇖2 ⇖1 ⇓2 ⇓3 ⇑1 ⇘3 ⇖2 ⇘2 ⇑2 ⇓2 ⇘1 ⇘3 ⇖1 ⇐3 ⇒3 ⇑1 ⇘3 ⇑2 ⇐3 ⇖1 ⇙1 ⇐3

In [1]:
# Make sure the output dir is available
!mkdir abram.gen.out

mkdir: cannot create directory ‘abram.gen.out’: File exists


In [2]:
# -*- coding: utf-8 -*-
# UTF8 hacks invoked.
import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
#reload(sys)
import codecs
#UTF8Writer = codecs.getwriter('utf8')
#sys.stdout = UTF8Writer(sys.stdout)
import util
import argparse
import inkey
import music21
import copy
import itertools
import re
import json
import time

n_dirs = 8
n_bumps = 3
directions = {
    "⇑":0,
    "⇗":1,
    "⇒":2,
    "⇘":3,
    "⇓":4,
    "⇙":5,
    "⇐":6,
    "⇖":7
}
inverse_directions = dict([(directions[key],key) for key in directions])
dir_string = "".join([inverse_directions[x] for x in range(0,n_dirs)])

def token_to_dir_tuple(token):
    return (directions[token[0:-1]], int(token[-1]))

def dir_tuple_to_string(token):
    return inverse_directions[token[0]]+str(token[1])

def load_transcript(filename):
    tokens = util.load_split_file(filename)
    return [token_to_dir_tuple(token) for token in tokens]

def rotate_list(l):
    return l[1:] + [l[0]]

def n_rotated_lists(l,n):
    out = range(0,n)
    out[0] = l
    for i in range(1,n):
        out[i] = rotate_list(out[i-1])
    return out

full_note      = 4.0
half_note      = 2.0
quarter_note   = 1.0
eighth_note    = 0.5
sixteenth_note = 0.25
note_durations = [full_note, half_note, quarter_note, eighth_note, sixteenth_note]

def make_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right
    return new_note

def pitches_of_key(mkey):
    m = re.match('([A-G][-#]?)(major|minor)',mkey)
    k = m.group(1)
    maj = m.group(2)
    our_key = music21.key.Key(k,maj)
    our_pitches = our_key.pitches
    return our_pitches

def generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave):
    """ returns a list of music21 notes """
    our_pitches = pitches_of_key(mkey)
    # print(our_pitches)
    # modify the key to the appropriate octave
    our_key_notes = [inkey.note_octave_mod(note, octave-4) for note in our_pitches]
    # now get the note mapping to notes
    our_notes = [our_key_notes[x] for x in note_mapping]
    # process the tuple
    # print song_tuples
    # print bump_mapping
    assert sum([len(t)==2 for t in song_tuples]) == len(song_tuples)
    # print [ (our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]
    return [ make_note(our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]

def note_to_str(note):
    return note.fullName + " " + str(note.duration) #music21.Note.fullName(note)
def song_to_txt(song):
    return "\n".join([note_to_str(note) for note in song])

def save_textfile(txt, filename):
    fd = file(filename,"w")
    fd.write(txt)
    fd.close()

def song_to_mid(song):
    stream = music21.stream.Stream()
    for elm in song:
        stream.append(elm)
    return stream
    # mf = music21.midi.translate.streamToMidiFile(stream)

def save_midifile(midi, midi_file):
    fp = midi.write('midi', fp=midi_file)
    
def strify(l):
    return [str(x) for x in l]
    
def generate_and_save_song(song_tuples,mkey, note_mapping, bump_mapping,octave):
    song = generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave)
    file_name = "abram.gen.out/" + mkey + ".O." + str(octave) + ".N." + ".".join("".join(strify(note_mapping))) + ".T." + "_".join(strify(bump_mapping))
    txt_file_name = file_name + ".txt"
    mid_file_name = file_name + ".mid"
    txt = song_to_txt(song)
    save_textfile(txt, txt_file_name)
    # print(song)
    mid = song_to_mid(song)
    save_midifile(mid, mid_file_name)

__grid_search_print__ = True
def grid_search(params,param_keys,song_tuples,f=None,maxtime=None):
    if maxtime is None:
        maxtime = float("inf")
    start = time.time()
    iters = 0
    for t in itertools.product(*[params[x] for x in param_keys]):
        if __grid_search_print__:
            print(t)
        f(song_tuples,*t)
        iters += 1
        if (time.time() - start > maxtime):
            print("Times up! %s" % iters)
            break
    return iters

def my_main(maxtime=10):
    filename = "abram-dirs.txt"
    song_tuples = load_transcript(filename)
    print(inkey.get_all_keys().keys())
    #print("\t".join([dir_tuple_to_string(x) for x in our_tokens]))
    params = {
        "keys":inkey.get_all_keys().keys(),
        # permutations of indices
        #  could do all permutations too
        "dir_notes":n_rotated_lists(range(0,n_dirs),n_dirs),
        # "dir_notes":list(itertools.permutations(range(0,n_dirs),n_dirs)) # if you want 40k combos
        # map bumps to duration
        "bump_durations":list(itertools.permutations(note_durations,n_bumps)),
        "octaves":range(3,6),
    }
    return grid_search(params,["keys","dir_notes","bump_durations","octaves"], 
                song_tuples, f=generate_and_save_song, maxtime=maxtime)

def generate_and_estimate_song(estimate,song_tuples,mkey, note_mapping, bump_mapping,octave):
    song = generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave)
    file_name = "abram.gen.out/" + mkey + ".O." + str(octave) + ".N." + ".".join("".join(strify(note_mapping))) + ".T." + "_".join(strify(bump_mapping))
    txt = song_to_text(song)
    return estimate(txt), file_name
    
def my_estimate_all(estimator, transcription_song_tuples):
    estimates = dict()
    def helper(estimator,song_tuples,mkey, note_mapping, bump_mapping,octave):
        estimate, name = generate_and_estimate_song(estimator,song_tuples,mkey, note_mapping, bump_mapping,octave)
        estimates[name] = estimate
        print((estimate, name))
    
    params = {
        "keys":inkey.get_all_keys().keys(),
        # permutations of indices
        #  could do all permutations too
        # "dir_notes":n_rotated_lists(range(0,n_dirs),n_dirs),
        "dir_notes":list(itertools.permutations(range(0,n_dirs),n_dirs)), # if you want 40k combos
        # map bumps to duration
        "bump_durations":list(itertools.permutations(note_durations,n_bumps)),
        "octaves":range(3,6),
    }
    grid_search(params,["keys","dir_notes","bump_durations","octaves"], song_tuples, f=helper)
    pickle.dump(estimates, file('abram-song-estimates.pkl','w'))    

def my_estimate(args):
    filename = args.transcript
    song_tuples = load_transcript(filename)
    my_estimate_all(None,song_tuples)
    
def test_rotate_list():
    assert rotate_list([0,1,2,3]) == [1,2,3,0]
    assert rotate_list([0,1,2,3,4]) == [1,2,3,4,0]
    assert rotate_list(range(0,100))[0:-1] == range(1,100)
    input  = [0,1,2]
    output = [[0,1,2],[1,2,0],[2,0,1]]
    assert json.dumps(n_rotated_lists(input, 3)) == json.dumps(output)    

def test_notes():
    note = music21.pitch.Pitch("C#4") 
    new_note = make_note( note, quarter_note )
    assert new_note.duration.type == 'quarter'
    assert new_note.pitch.fullName == 'C-sharp in octave 4'    
    song = song_to_mid([copy.deepcopy(new_note) for i in range(0,10)])
    notes = list(song.notesAndRests)
    for i in range(0,len(notes)):
        assert notes[i].duration.type == new_note.duration.type
        assert notes[i].pitch.fullName == new_note.pitch.fullName
    cmajor = pitches_of_key("Cmajor")
    assert [x.name for x in cmajor] == ["C","D","E","F","G","A","B","C"]
    
def my_tests(args):
    test_rotate_list()
    test_notes()
    


In [3]:
my_main(5)


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

70

In [4]:
seconds = 1
repeats = 10
def mean(x):
    return sum(x)/len(x)
perf = map(lambda x: my_main(seconds),range(0,repeats))
perf

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 4)
Times up! 14
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.

[15, 14, 14, 15, 14, 14, 15, 14, 14, 15]

In [5]:
perf
print("Current Performance ~ %s iters per %s seconds " % (mean(perf), seconds))

Current Performance ~ 14 iters per 1 seconds 


4. Original performance? 
   * 14 song per second? 
   * That's really very poor.
5. Make sure we are optimizing what want optimized? 
   * Time.
   * But at least we know what we want: More iters per second.
6. Record original performance
   * 14 song per second
7. Investigate performance
   * We're going to use a profiler
8. Change system
   * not until we profile!

At this point we either have a good idea of what is slow or we need some help to find out what.

Profilers instrument running code and take statistics about the number of calls and the 

Checkout https://docs.python.org/2/library/profile.html

In [6]:
import cProfile

In [7]:
cProfile.run('my_main(4)','cpout.txt')

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

In [8]:
import pstats

In [9]:
p = pstats.Stats('cpout.txt')
p.strip_dirs().sort_stats(-1).print_stats()

Sun May 12 23:32:33 2019    cpout.txt

         5219482 function calls (5092334 primitive calls) in 4.024 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       38    0.002    0.000    0.107    0.003 <ipython-input-2-50c167bfa7f6>:100(song_to_mid)
       38    0.000    0.000    2.031    0.053 <ipython-input-2-50c167bfa7f6>:107(save_midifile)
       76    0.000    0.000    0.000    0.000 <ipython-input-2-50c167bfa7f6>:110(strify)
       38    0.001    0.000    4.022    0.106 <ipython-input-2-50c167bfa7f6>:113(generate_and_save_song)
        1    0.001    0.001    4.026    4.026 <ipython-input-2-50c167bfa7f6>:124(grid_search)
        1    0.000    0.000    4.026    4.026 <ipython-input-2-50c167bfa7f6>:138(my_main)
       87    0.000    0.000    0.000    0.000 <ipython-input-2-50c167bfa7f6>:35(token_to_dir_tuple)
        1    0.000    0.000    0.000    0.000 <ipython-input-2-50c167bfa7f6>:41(load_transcript)
        7    0.000

<pstats.Stats instance at 0x7f6d60ecfd88>

In [10]:
p.sort_stats('cumulative').print_stats(10)

Sun May 12 23:32:33 2019    cpout.txt

         5219482 function calls (5092334 primitive calls) in 4.024 seconds

   Ordered by: cumulative time
   List reduced from 420 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.026    4.026 <string>:1(<module>)
        1    0.000    0.000    4.026    4.026 <ipython-input-2-50c167bfa7f6>:138(my_main)
        1    0.001    0.001    4.026    4.026 <ipython-input-2-50c167bfa7f6>:124(grid_search)
       38    0.001    0.000    4.022    0.106 <ipython-input-2-50c167bfa7f6>:113(generate_and_save_song)
       38    0.000    0.000    2.031    0.053 <ipython-input-2-50c167bfa7f6>:107(save_midifile)
       38    0.000    0.000    2.031    0.053 __init__.py:246(write)
       38    0.000    0.000    2.031    0.053 base.py:2529(write)
       38    0.000    0.000    2.021    0.053 subConverters.py:921(write)
       38    0.000    0.000    1.963    0.052 translate.py:237(mu

<pstats.Stats instance at 0x7f6d60ecfd88>

In [11]:
p.sort_stats('tottime').print_stats(10)

Sun May 12 23:32:33 2019    cpout.txt

         5219482 function calls (5092334 primitive calls) in 4.024 seconds

   Ordered by: internal time
   List reduced from 420 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    29374    0.216    0.000    0.515    0.000 collections.py:50(__init__)
119928/2850    0.157    0.000    1.370    0.000 copy.py:145(deepcopy)
     3344    0.103    0.000    0.934    0.000 base.py:479(_deepcopySubclassable)
    25308    0.090    0.000    0.590    0.000 base.py:409(__init__)
     7676    0.090    0.000    0.133    0.000 pitch.py:2309(_setName)
    29374    0.086    0.000    0.250    0.000 _abcoll.py:548(update)
    53466    0.079    0.000    0.079    0.000 collections.py:71(__setitem__)
    13452    0.075    0.000    0.079    0.000 __init__.py:485(__init__)
    19836    0.074    0.000    0.143    0.000 sites.py:330(add)
    28652    0.070    0.000    0.581    0.000 sites.py:167(__init__)




<pstats.Stats instance at 0x7f6d60ecfd88>

Above, do you notice what library is taking up most of the top 6 places?

2 of those calls are to init, so that's object creation.

There's also a reference to deepcopy.

In [12]:
p.sort_stats('ncalls').print_stats(20)

Sun May 12 23:32:33 2019    cpout.txt

         5219482 function calls (5092334 primitive calls) in 4.024 seconds

   Ordered by: call count
   List reduced from 420 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   677160    0.039    0.000    0.039    0.000 {id}
   226100    0.043    0.000    0.045    0.000 {getattr}
   199191    0.056    0.000    0.131    0.000 {isinstance}
   190114    0.022    0.000    0.022    0.000 {method 'append' of 'list' objects}
   182020    0.025    0.000    0.025    0.000 {method 'get' of 'dict' objects}
   134102    0.025    0.000    0.026    0.000 {setattr}
126464/126388    0.014    0.000    0.014    0.000 {len}
119928/2850    0.157    0.000    1.370    0.000 copy.py:145(deepcopy)
    94316    0.024    0.000    0.024    0.000 {round}
    86678    0.044    0.000    0.044    0.000 _weakrefset.py:70(__contains__)
    81691    0.035    0.000    0.054    0.000 weakrefTools.py:49(unwrapWeakref)
    80370 

<pstats.Stats instance at 0x7f6d60ecfd88>

In [13]:
# here's a candidate function that creates lots of music21 objects.

def make_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right
    return new_note


Problem: Makes too many objects that are for some reason very slow to create.
    
These are notes, with set duration. Luckily how I use these notes is immutable. I do not change them.

Therefore I can SHARE notes if they are already created. That is I can memoize the output of this function!

In [14]:
# keep the old code to really make it
def _make_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right
    return new_note

# a place to cache the created notes
__note_cache__ = dict()
# memoize
def make_note( midi21_pitch, duration ):
    # make a key
    note_key = str(midi21_pitch)+str(duration)
    # if the note isn't in the cache make it
    if not note_key in __note_cache__:
        __note_cache__[note_key] = _make_note(midi21_pitch, duration)
    return __note_cache__[note_key]

In [15]:
def bench(seconds=1,repeats=10):
    perf = map(lambda x: my_main(seconds),range(0,repeats))
    print("Current Performance ~ %s iters per %s seconds " % (mean(perf), seconds))
    print(perf)

In [16]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)


StreamException: the object (<music21.note.Note D#>, id()=140107754052560) is already found in this Stream (<music21.stream.Stream 0x7f6d60ca9a10>, id()=140107752053264)

In [17]:
def song_to_mid(song):
    stream = music21.stream.Stream()
    for elm in song:
        note = music21.note.Note()
        note.pitches = elm.pitches
        note.duration = elm.duration
        stream.append(note)
    return stream

In [18]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 3)
Times up! 16
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor',

In [19]:
cProfile.run('my_main(4)','cpout2.txt')


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

In [20]:

p = pstats.Stats('cpout2.txt')
p.sort_stats('tottime').print_stats(10)
p.sort_stats('cumtime').print_stats(20)


Sun May 12 23:33:26 2019    cpout2.txt

         5148243 function calls (5023575 primitive calls) in 4.071 seconds

   Ordered by: internal time
   List reduced from 425 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    30159    0.230    0.000    0.541    0.000 /usr/lib/python2.7/collections.py:50(__init__)
124020/2925    0.158    0.000    1.341    0.000 /usr/lib/python2.7/copy.py:145(deepcopy)
    11271    0.126    0.000    0.189    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/pitch.py:2309(_setName)
     3432    0.109    0.000    0.880    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:479(_deepcopySubclassable)
    25986    0.094    0.000    0.673    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:409(__init__)
    30159    0.093    0.000    0.260    0.000 /usr/lib/python2.7/_abcoll.py:548(update)
    80208    0.084    0.000    0.084    0.000 /home/hindle1/.local/

<pstats.Stats instance at 0x7f6d609e91b8>

I saw this:
    
    40    0.000    0.000    1.475    0.037 <ipython-input-20-50c167bfa7f6>:67(pitches_of_key)

In [21]:
__pitches_of_key__ = dict()
def _pitches_of_key(mkey):
    m = re.match('([A-G][-#]?)(major|minor)',mkey)
    k = m.group(1)
    maj = m.group(2)
    our_key = music21.key.Key(k,maj)
    our_pitches = our_key.pitches
    return our_pitches

# memoize
def pitches_of_key(mkey):
    if not mkey in __pitches_of_key__:
        __pitches_of_key__[mkey] = _pitches_of_key(mkey)
    return __pitches_of_key__[mkey]



In [22]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4

Boom. Performance Improved! 14 to 23. Good. 33% faster

In [23]:
cProfile.run('my_main(4)','cpout3.txt')
p = pstats.Stats('cpout3.txt')
p.sort_stats('tottime').print_stats(10)
p.sort_stats('cumtime').print_stats(20)


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

<pstats.Stats instance at 0x7f6d608d3950>

In [24]:
p.sort_stats('ncalls').print_stats(30)

Sun May 12 23:34:01 2019    cpout3.txt

         5807919 function calls (5634057 primitive calls) in 4.045 seconds

   Ordered by: call count
   List reduced from 331 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   997845    0.055    0.000    0.055    0.000 {id}
   254589    0.027    0.000    0.027    0.000 {method 'append' of 'list' objects}
   249222    0.034    0.000    0.034    0.000 {method 'get' of 'dict' objects}
   245007    0.050    0.000    0.053    0.000 {getattr}
   186163    0.046    0.000    0.086    0.000 {isinstance}
169116/600    0.199    0.000    1.922    0.003 /usr/lib/python2.7/copy.py:145(deepcopy)
   151026    0.030    0.000    0.030    0.000 {setattr}
   126900    0.023    0.000    0.023    0.000 {method 'startswith' of 'str' objects}
   115506    0.037    0.000    0.037    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/common/weakrefTools.py:18(wrapWeakref)
   104340    0.089    0.000    0

<pstats.Stats instance at 0x7f6d608d3950>

We're manipulating the objects alot with their octaves.

In [25]:
# memoize
def pitches_of_key(mkey):
    if not mkey in __pitches_of_key__:
        __pitches_of_key__[mkey] = _pitches_of_key(mkey)
    return __pitches_of_key__[mkey]

__pitches_octave_mod__ = dict()
def pitch_octave_mod( pitch, octave ):
    mkey = str(pitch)+str(octave)
    #mkey = (pitch,octave)
    if not mkey  in __pitches_of_key__:
        __pitches_octave_mod__[mkey] = inkey.note_octave_mod(pitch, octave)
    return __pitches_octave_mod__[mkey]



# memoize
def _get_pitches_of_key_with_octave(mkey, octave):
    our_pitches = pitches_of_key(mkey)
    our_key_notes = [pitch_octave_mod(note, octave-4) for note in our_pitches]
    return our_key_notes

# memoize
__key_and_octave__ = dict()
def get_pitches_of_key_with_octave(mkey, octave):
    our_key = "%s %s" % (mkey,str(octave))
    if not our_key in __key_and_octave__:
        __key_and_octave__[our_key] = _get_pitches_of_key_with_octave(mkey, octave)
    return __key_and_octave__[our_key]    

def generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave):
    """ returns a list of music21 notes """
    # our_pitches = pitches_of_key(mkey)
    # # print(our_pitches) 
    # # modify the key to the appropriate octave
    # our_key_notes = [pitch_octave_mod(note, octave-4) for note in our_pitches]
    our_key_notes = get_pitches_of_key_with_octave(mkey, octave)
    # now get the note mapping to notes
    our_notes = [our_key_notes[x] for x in note_mapping]
    # process the tuple
    # print song_tuples
    # print bump_mapping
    assert sum([len(t)==2 for t in song_tuples]) == len(song_tuples)
    # print [ (our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]
    return [ make_note(our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]



In [26]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4

No real gain from that.

In [27]:
cProfile.run('my_main(4)','cpout4.txt')


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

In [28]:
p = pstats.Stats('cpout4.txt')
# p.sort_stats('cumtime').print_stats(25)
p.strip_dirs().sort_stats('cumtime').print_stats(50)

Sun May 12 23:34:37 2019    cpout4.txt

         5384352 function calls (5223896 primitive calls) in 4.068 seconds

   Ordered by: cumulative time
   List reduced from 324 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.068    4.068 <string>:1(<module>)
        1    0.000    0.000    4.068    4.068 <ipython-input-2-50c167bfa7f6>:138(my_main)
        1    0.002    0.002    4.067    4.067 <ipython-input-2-50c167bfa7f6>:124(grid_search)
       56    0.001    0.000    4.060    0.073 <ipython-input-2-50c167bfa7f6>:113(generate_and_save_song)
       56    0.000    0.000    3.224    0.058 <ipython-input-2-50c167bfa7f6>:107(save_midifile)
       56    0.001    0.000    3.224    0.058 __init__.py:246(write)
       56    0.001    0.000    3.223    0.058 base.py:2529(write)
       56    0.001    0.000    3.207    0.057 subConverters.py:921(write)
       56    0.000    0.000    3.106    0.055 translate.py:237(m

<pstats.Stats instance at 0x7f6d60efebd8>

In [33]:
__key_and_octave__ = dict()
__pitches_of_key__ = dict()
__pitches_octave_mod__ = dict()

def _make_note( midi_pitch, duration ):
    return (midi_pitch, duration)
def make_note( midi_pitch, duration ):
    return _make_note(midi_pitch, duration)
def _get_pitches_of_key_with_octave(mkey, octave):
    our_pitches = pitches_of_key(mkey)
    our_key_notes = [pitch_octave_mod(note, octave-4) for note in our_pitches]
    # convert to midi
    return [p.midi for p in our_key_notes]
    # return our_key_notes
def note_to_midi(note):
    # if not note in __note_to_midi__:
    #     __note_to_midi__[note] = note.pitch.midi # supposedly this is slow
    # return __note_to_midi__[note]
    return int(note[0])

def song_to_txt(song):
    # return " ".join(["M{:03d}".format(note.midi) for note in song])
    return " ".join(["M{:03d}".format(note_to_midi(note)) for note in song])


In [34]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)


AttributeError: 'tuple' object has no attribute 'pitches'

In [37]:
def make_music21_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right    
    return new_note

def song_to_mid(song):
    stream = music21.stream.Stream()
    for elm in song:
        if isinstance(elm, music21.note.Note):
            #note = music21.note.Note()
            #note.pitches = elm.pitches
            #note.duration = elm.duration
            stream.append(elm)
        else:
            stream.append(make_music21_note(elm[0],elm[1]))
    return stream


In [38]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 0.5, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 0.5, 2.0), 4)
Times up! 20
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.

In [40]:
cProfile.run('my_main(4)','cpout5.txt')


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

In [42]:
p = pstats.Stats('cpout5.txt')
# p.sort_stats('cumtime').print_stats(25)
p.sort_stats('cumtime').print_stats(50)

Sun May 12 23:43:29 2019    cpout5.txt

         5554579 function calls (5348829 primitive calls) in 4.056 seconds

   Ordered by: cumulative time
   List reduced from 315 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.056    4.056 <string>:1(<module>)
        1    0.000    0.000    4.056    4.056 <ipython-input-2-50c167bfa7f6>:138(my_main)
        1    0.002    0.002    4.056    4.056 <ipython-input-2-50c167bfa7f6>:124(grid_search)
       50    0.001    0.000    4.049    0.081 <ipython-input-2-50c167bfa7f6>:113(generate_and_save_song)
       50    0.000    0.000    3.381    0.068 <ipython-input-2-50c167bfa7f6>:107(save_midifile)
       50    0.000    0.000    3.380    0.068 /home/hindle1/.local/lib/python2.7/site-packages/music21/stream/__init__.py:246(write)
       50    0.001    0.000    3.380    0.068 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:2529(write)
       50    0.00

<pstats.Stats instance at 0x7f6d6092d950>

In [43]:
It's still the midi output.

SyntaxError: EOL while scanning string literal (<ipython-input-43-4d3b53a5681a>, line 1)

In [59]:
__grid_search_print__ = False

def save_midifile(x,y):
    return None
def song_to_mid(x):
    return None

In [60]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7506
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7805
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7997
['Em

In [61]:
cProfile.run('my_main(4)','cpout6.txt')
p = pstats.Stats('cpout6.txt')
p.sort_stats('cumtime').print_stats(50)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 19689
Sun May 12 23:56:24 2019    cpout6.txt

         9150707 function calls (9148343 primitive calls) in 4.001 seconds

   Ordered by: cumulative time
   List reduced from 225 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.001    4.001 <string>:1(<module>)
        1    0.000    0.000    4.001    4.001 <ipython-input-2-50c167bfa7f6>:138(my_main)
        1    0.070    0.070    4.000    4.000 <ipython-input-58-7fe5e4b4af6a>:2(grid_search)
    19689    0.091    0.000    3.928    0.000 <ipython-input-2-50c167bfa7f6>:113(generate_and_save_song)
    19689    0.670    0.000    1.470 

<pstats.Stats instance at 0x7f6d60da37a0>

Well that's neat. But maybe we should do something else? Like actually save midi files to disk?

In [65]:
# remove the function call to _make_note
def make_note( midi_pitch, duration ):
    return (midi_pitch, duration)
bench()
cProfile.run('my_main(4)','cpout7.txt')
p = pstats.Stats('cpout7.txt')
p.sort_stats('cumtime').print_stats(20)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8569
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8541
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8458
['Em

<pstats.Stats instance at 0x7f6d60a53e60>

In [66]:
# remove the function call to _make_note
from collections import defaultdict
__make_note__ = defaultdict(dict)
def make_note( midi_pitch, duration ):
    if not duration in __make_note__[midi_pitch]:
        __make_note__[midi_pitch][duration] = (midi_pitch, duration)
    return __make_note__[midi_pitch][duration]

bench()
cProfile.run('my_main(4)','cpout8.txt')
p = pstats.Stats('cpout8.txt')
p.sort_stats('cumtime').print_stats(20)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7043
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7683
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7580
['Em

<pstats.Stats instance at 0x7f6d60d7c4d0>

In [67]:
# nope memoize did not help
def make_note( midi_pitch, duration ):
    return (midi_pitch, duration)
